# HW2

Dylan Ang

For ECS 171 - Fall 2022 - E. Solares

### Download the data
Let's download and uncompress our data and images here:

In [ ]:
# We need to first download the data here:
file_id = '1-mQuJbHvSq9nRugy96dr0aB7isfzaxMZ'
file_download_link = "https://docs.google.com/uc?export=download&id=" + file_id
!wget -O data.zip --no-check-certificate "$file_download_link"
!unzip data.zip

# Artificial Neural Networks


### Contents
1. Artificial neural networks (ANNs) and their link to Logistic regression
2. Generalization of multi layer networks
3. Step by step implementation of our ANN class
4. Does it do what it is supposed to do?
    1. linear regression: 25 all over again!
    2. Another go at our Titanic dataset
    3. Complex fit of flowers
    4. Andrew Ng's cat vs non-cat classifying
5. Round up
**Apendix:**
- Data creation/processing for Rose / Cats
- Crude version of a hyperparameter sweep



## Artificial neural networks (ANNs) and their link to Logistic regression

When we would ask a random person about Machine Learning, there is a big chance that neural networks are mentioned. Not only does the terminology play with our imagination, but these mathematical structures have also proven themselves to solve complex tasks. Many of you have probably seen such a network and it almost radiates simplicity:

<img src="https://drive.google.com/uc?id=10yZYlGrKUh0uFDQ862TphkNjT3EBQBIS" alt="Artificial Neural Network example" width="500" style="display: block; margin: 0 auto" />


In this simple diagram, we show an artificial neural network (ANN), or just simply neural network (NN), which has three layers. By convention, we do not count the input layer, and later we see that this layer in the diagram represents the input data you feed into the NN. This particular NN has two hidden layers. While I am not sure why it exactly is called a hidden layer, I can imagine that one reason could be that these layers are 'hidden' from the user. A user inputs data through the input data and gets results from the output layer, therefore, not interacting with the hidden layers. The user would not see if there is just a single layer, two layers, or even ten layers. The output however, is an actual layer, in our case a single neuron, which 'collects' the results from the previous hidden layer.


A logistic regression can be seen as the tiniest possible NN, with just a single layer, consisting of a single neuron. A diagram of a logistic regression can look like this:

<img src="https://drive.google.com/uc?id=1W-qiH9eroJJUZR82Qy81liXnBK_knp37" alt="Artificial Neural Network example" width="400" style="display: block; margin: 0 auto" />

Let's do a short recap on this logistic model. First, we massaged the input data in such a way that the individual features ($x_1, x_2, x_3$) are in the rows of our input vector $X$, and the columns are the examples (training samples). This input vector $X$ is then forwarded to our single neuron, which can be divided in two parts. First, a linear operation (linear regression), $z = X W + b$, followed by an activation function $A = g(z)$. In our previous tutorial we performed binary logistic regression, which uses the sigmoid activation function $\sigma(z) = \frac{1}{1 + e^{-z}}$. Here we generalize this by using $g(z)$ to indicate an activation function we have to define. As we will learn today, there are many other activation functions to choose from.

Before we can calculate the activation function we need to calculate the linear part $z$. This is the inner product of the weights vector $W$ and the input vector $X$, with the bias term $b$ added. The bias term $b$ and the weights vector $W$ (consisting of $w_1, w_2, w_3$) are the trainable parameters of this system. Each trainable weight ($w_1, w_2, w_3$) corresponds to an input feature ($x_1, x_2, x_3$) and represents the 'weight' this feature is adding to the problem. These weights are drawn inside the output layer frame, meaning that they are linked to this layer (inside this single neuron).


While logistic regression is a great tool, in can only divide the parameter space in a line, at least in the from we have presented here. For example, if you have two features $x_1$ and $x_2$, which will be used to predict $y$, a logistic classifier is only able to have a linear boundary between the two parameters. If this is not completely clear, we will have an example to show this problem.

To have the system predict more complex relations, we can add more neurons to a layer and even add more layers to our network. Each of this neuron is a kind of logistic regression unit and many of these combined can predict highly non-linear relations. I say kind of, because in a logistic regression we typically use the sigmoid function, while in a NN, many other activation function perform much better.

<img src="https://drive.google.com/uc?id=1E1iyQlEDcd7ARsnTS-Tgm3JaK3lGwfKB" alt="Artificial Neural Network example" width="500" style="display: block; margin: 0 auto" />

Before we will start to generalize a NN and the used layers, observe how the different neurons are connected. You will notice that each node is connected to all nodes of the next layer. This is called densely connected (sometimes fully connected) and such a layer is often called a 'Dense layer'. Now, lets try to define such structures.


## Generalization of multi layer networks

Before we can start to develop our system, lets try to first generalize the steps required. The goal of this tutorial is to create a general NN class in which we can add arbitrary number of layers, containing arbitrary number of neurons. Later we will test this structure on various problems with varying complexity.

When we think of the input layer, we have already discovered that it is not an actual layer of the neural network, but the training data, massaged in the right form. Therefore, the input 'layer' is not part of our architecture.

The neural network will however consist of an arbitrary number of layers. These will be structures that behave in a similar fashion and are sequetial to each other. When doing the forward pass, we will loop through all the layers and use the output of the previous layer as input for the current layer. The output of the final layer, also called the final activation $A$ is the output of the neural network. This means that, if the neural network is used to predict binary values, the output has to be converted (or rounded) to the actual predictions $\hat Y$.

To do one pass of gradient descent, we need to have the weights. For this, we need to calculate the gradient of the loss function with respect of $W$ and $b$ for each layer. To calculate the gradients, we can do a clever trick and cache the values of $z$ and $A$ during the forward pass. We need to do some bookkeeping here the most mathematically heavy part is quite doable.

After all gradients are calculated the weights are then updated using a simple gradient descent step. To make is visual I have created the following diagram:

<img src="https://drive.google.com/uc?id=1jLFZ28_VOCKXmYkaIhVK070gNOuQOSf7" alt="Artificial Neural Network example" width="700" style="display: block; margin: 0 auto" />

Here the forward pass calculates our loss and is used in the backward pass to calculate updated weights based on alpha. Next we calculate our loss again in the forward pass which then is used again in the next GD step to update weights based on the gradient descent step based on alpha. We do this until we minimize our loss past a certain threshold or after a certain number of iterations. We will now go into detail of these steps.

All layers in the network are fundamentally the same. They can differ in the amount of nodes or the activation function, but there is no structural difference between the first layer $L_1$ and the output layer $L_o$. 

As we will be creating only fully connected Dense layers, we can create a single structure to do the hard work. In this structure we have to do the math for the forward pass, the backward pass, and the update. The input of each layer is generally the output of the previous layer. Obviously, for the first layer, this is the input vector. For the backward pass, we sequentially go through the network in the reverse direction. However, here we need to do an addtional step. We need to calculate the gradients with respect to the defined loss function. For this we need to input the true labels $Y$ and the predicted output $A$ during the backward pass. After this, we again have that the previous gradient is the input for the next layer (remember: we are going backwards). Each layer handels the backwards pass in the same way:

<img src="https://drive.google.com/uc?id=13IN6rIkiSdCuT8lT8BSasCj4wdA_spGW" alt="Artificial Neural Network example" width="600" style="display: block; margin: 0 auto" />


For each layer, in the forward pass, in expects the output of the previous layer (or $X$). This is then cached for use during the backward pass. We calculate the linear part $z$ with one single vectorized inner product calculation for all neurons and add the bias term. We also cache this value of $z$ for the backward pass. Finally, we will calculate the activation function and pass the result to the next layer (or this is the output if it is the final layer).

The backward pass is just the forward pass in reverse, but expects the previous gradient as an input. We calculate the gradients, using the cached values of $A_p$ and $z_p$. As a last step we calculate $dA_p$ which will be the input to the next layer. I do not want to focus to much on the actual differentials, but will explain them a bit more in the code. If you really want to know how these differentials are calculated, I would suggest to get a pen and paper and try to calculate them. They are not hard and [Wolfram Alpha](https://www.wolframalpha.com/) can help you :-).


If it is not completely clear what each step is doing, do not worry to much about it. I will try to explain each step during the coding part. To have an idea, let us summarize what we will be making:

#### Dense Layer Class
- structure to hold an arbitrary number of nodes
- will have various activation functions
- will perform the forward pass through the single layer (z and A)
- will perform the backward pass for the layer
- will perform the update step for the layer

#### Neural Network Class
- structure to hold an arbitrary number of layers
- will perform the forward pass sequentially through all layers
- will do the cost calculation for various loss functions
- will perform the backward pass and calculate all gradients
- will do an update (a step of gradient descent)


## Step by step implementation of our ANN class

Now lets start to implement our NN classes. First import numpy

In [ ]:
import numpy as np

Before we can define our main NN class, we need to define our Dense layer class. As I want to do some error checking, we will first define some custom Exceptions:

In [ ]:
class ActivationDoesNotExist(Exception):
    """Valid activations are sigmoid, tanh, and relu, provided as a string"""
    pass

class InputDimensionNotCorrect(Exception):
    """Need to specify input dimension, i.e. input shape into the first layer"""
    pass

class LossFunctionNotDefined(Exception):
    """Loss function in cost() method not defined"""
    pass

These exception extend Python's default Exception class and now can be used with the 'raise' keyword to create an exception that tells the user what the mistake is. If you systems get larger, it is good practice to have meaningfull errors. Another great way is using the logging module.

Now we can begin with our new class called DenseLayer. This class takes an constructor with two required parameters:
- inputDimension, i.e. the number of features of the input vector, or the number of units from the previous layer
- units, i.e. the number of neurons in this layer
There are two more optional parameters:
- activation: here you can indicate what activation the layer should use. Defined are sigmoid, relu, tanh, and ''. The empty string indicates no activation, meaning we just have a regression.
- randomMultiplier is the value which the random weights are multiplied. Generally 0.01 is fine, but sometimes tweaking this number can help.

As there can me different activation functions and we do not want to chech which activation we have using an if-statement, we make a reference to the used activation functions in the init statement.

In the initialize method, the weights are initialized. Notice that the number of neurons nh (units) are in the rows, and the number of input features nx are the columns. This is required to make our dot product work later.

Another thing I learned recently is the use of 'self' in Python. While I though I understood the concept, I did not fully understand the concequences. The definition of the class is separated from the values of each instance. These values are stored in the self object, i.e. the object of the instance. In other programming languages it is common to define types it the class itself. However, in Python you have to define these in the init method. When not doing this, the variable is the same object in all your instances and you can get weird results. I just found this [blog-post](https://towardsdatascience.com/python-pitfall-mutable-default-arguments-9385e8265422) where Don Cross has a very clear explanation.


```python
class DenseLayer:
    def __init__(self, inputDimension, units, activation='', randomMultiplier=0.01):
        self.weights, self.bias = self.initialize(inputDimension, units, randomMultiplier)
        if activation == 'sigmoid':
            self.activation = activation
            self.activationForward = self.sigmoid
            self.activationBackward = self.sigmoidGrad
        elif activation == 'relu':
            self.activation = activation
            self.activationForward = self.relu
            self.activationBackward = self.reluGrad
        elif activation == 'tanh':
            self.activation = activation
            self.activationFunction = self.tanh
            self.activationBackward = self.tanhGrad
        elif activation != '':
            raise ActivationDoesNotExist
        else:
            self.activation = 'none'
            self.activationFunction = self.linear
            self.activationBackward = self.linear
    
    def initialize(self, nx, nh, randomMultiplier):
        weights = randomMultiplier * np.random.randn(nh, nx)
        bias = np.zeros([nh, 1])
        return weights, bias

```

Next we define all used activation functions. The sigmoid we already know from the previous Tutorial, however we also include tanh and relu, which are also very common activation functions.

Tanh or hyperbolic tangent in full is a function which is similar to the sigmoid, however it maps all real values between -1 and +1. It was very popular, before the Relu function made its way to the stage. 

Relu, which stands for Rectified linear unit, is probably the most popular activation function. It is fast to calculate and has often better results than the Tanh function. Therefore, if you are not sure, the Relu function is a great start. The Relu function maps all values smaller than 0 to zero and all values larger than 0 as the value itself. 

The last activation function we will introduce is called the linear function. This is the same as not having an activation function, and is just a placeholder. What comes in, comes out and we use it to test our previous excersise on Linear regression.

The backward pass needs the differentials of all these functions, which are provided with the Grad suffix. Feel free to check if these differentials are correct. One additional step for each Grad calculation is that we multiply the input gradient (dA0 with the calculated gradient. This is required because we use the [chain rule](https://en.wikipedia.org/wiki/Differentiation_rules#The_chain_rule) (continously through back propagation).


```python
    def sigmoid(self, Z):
            A = 1 / (1 + np.exp(-Z))
            return A
        
    def sigmoidGrad(self, dA):
            s = 1 / (1 + np.exp(-self.prevZ))
            dZ = dA * s * (1 - s)
            return dZ
    
    
    def relu(self, Z):
            A = np.maximum(0, Z)
            return A
        
    def reluGrad(self, dA):
            s = np.maximum(0, self.prevZ)
            dZ = (s>0) * 1 * dA
            return dZ 

        
    def tanh(self, Z):
            A = np.tanh(Z)
            return A

    def tanhGrad(self, dA):
            s = np.tanh(self.prevZ)
            dZ = (1 - s**2) * dA
            return dZ


    def linear(self, Z):
        return Z
```

Next, we define the forward propagation step, which should look very familiar:
```python
    def forward(self, A):
        Z = np.dot(self.weights, A) + self.bias
        self.prevZ = Z
        self.prevA = A
        A = self.activationForward(Z)
        return A
```

We first calculate the linear part. Store the values for Z and A for later use in the back propagation and next apply the activation function.


Before starting our backpropstep we need to define our loss function. For logistic regression it's done with the binary cross entropy, for which I will simply give the equation. 
\begin{equation}
loss = -\frac{1}{m}\sum_{i=1}^{m}y\log(A)+(1-y)\log(1-A)
\end{equation}

Now for the backwards pass, We would need to differentiate the Loss function with $W$ and $b$. Not to  bore you guys, 
I have provided these functions:

\begin{equation}
 \frac{\partial loss}{\partial W} = \frac{1}{m} \sum_{i=1}^m  X(A - Y)^T \\
 \frac{\partial loss}{\partial b} = \frac{1}{m} \sum_{i=1}^m (A - y)
\end{equation}

In Python, this looks like this:

The backward propagation looks again similar to the previous tutorial, however we have split the backward activation part to the activation functions itself. Also this function expects to get dA, which is a more generalized form for multiple layers. In our previous single layer example we combined the differential of the loss function in this step. Now this step is in our NN class and not in each layer, as it is only required at the last layer.

```python
  def backward(self, dA):
        dZ = self.activationBackward(dA)
        m = self.prevA.shape[1]
        self.dW = 1 / m * np.dot(dZ, self.prevA.T)
        self.db = 1 / m * np.sum(dZ, axis=1, keepdims=True)
        prevdA = np.dot(self.weights.T, dZ)
        return prevdA 
```

The gradients are stored in the layer and can be used later by the update function, which performs the gradient descent step. This backward passes chain together for each layer as we will see later.


The final required function is the update function, which performs the gradient descent step. Nothing spectacular but it expects a learning rate.

```python
    def update(self, learning_rate):
        self.weights = self.weights - learning_rate * self.dW
        self.bias = self.bias - learning_rate * self.db
```


While not required, these next function help in printing the model

```python    
    def __repr__(self):
        act = 'none' if self.activation == '' else self.activation
        return f'Dense layer (nx={self.weights.shape[1]}, nh={self.weights.shape[0]}, activation={act})'
```


### The complete DenseLayer class

In [ ]:
class DenseLayer:
    """
    A class to define fully connected layers.
    """
    
    def __init__(self, inputDimension, units, activation='', randomMultiplier=0.01):
        """
        Constructor:
          inputDimension: number of input features
          units: number of neurons in the layer
          activation: activation function applied to layer
            - options: 'sigmoid', 'tanh', 'relu', ''
          randomMultiplier: multiplier applied to the random weights during initialization
        """
        self.weights, self.bias = self.initialize(inputDimension, units, randomMultiplier)
        if activation == 'sigmoid':
            self.activation = activation
            self.activationForward = self.sigmoid
            self.activationBackward = self.sigmoidGrad
        elif activation == 'relu':
            self.activation = activation
            self.activationForward = self.relu
            self.activationBackward = self.reluGrad
        elif activation == 'tanh':
            self.activation = activation
            self.activationForward = self.tanh
            self.activationBackward = self.tanhGrad
        elif activation != '':
            raise ActivationDoesNotExist
        else:
            self.activation = 'none'
            self.activationForward = self.linear
            self.activationBackward = self.linear
    
    def initialize(self, nx, nh, randomMultiplier):
        """
        Initializes weights randomly:
          nx: number of input features
          nh: number of units
          randomMultiplier: multiplier applied to the random weights during initialization
        returns:
          weights: the randomly initialized weights
          bias: the bias terms
        """
        weights = randomMultiplier * np.random.randn(nh, nx)
        bias = np.zeros([nh, 1])
        return weights, bias

    
    def sigmoid(self, Z):
        """
        Sigmoid activation function
        """
        A = 1 / (1 + np.exp(-Z))
        return A
        
    def sigmoidGrad(self, dA):
        """
        Differential of sigmoid function with chain rule applied
        """
        s = 1 / (1 + np.exp(-self.prevZ))
        dZ = dA * s * (1 - s)
        return dZ
    
    
    def relu(self, Z):
        """
        Relu activation function
        """
        A = np.maximum(0, Z)
        return A
        
    def reluGrad(self, dA):
        """
        Differential of relu function with chain rule applied
        """
        s = np.maximum(0, self.prevZ)
        dZ = (s>0) * 1 * dA
        return dZ 

        
    def tanh(self, Z):
        """
        Tanh activation function
        """
        A = np.tanh(Z)
        return A

    def tanhGrad(self, dA):
        """
        Differential of tanh function with chain rule applied
        """
        s = np.tanh(self.prevZ)
        dZ = (1 - s**2) * dA
        return dZ


    def linear(self, Z):
        """
        Placeholder when no activation function is used
        """
        return Z
        
    
    def forward(self, A):
        """
        Forward pass through layer
          A: input vector
        """
        Z = np.dot(self.weights, A) + self.bias
        self.prevZ = Z
        self.prevA = A
        A = self.activationForward(Z)
        return A
    
    
    def backward(self, dA):
        """
        Backward pass through layer
          dA: previous gradient
        """
        dZ = self.activationBackward(dA)
        m = self.prevA.shape[1]
        self.dW = 1 / m * np.dot(dZ, self.prevA.T)
        self.db = 1 / m * np.sum(dZ, axis=1, keepdims=True)
        prevdA = np.dot(self.weights.T, dZ)
        return prevdA
    
    
    def update(self, learning_rate):
        """
        Update weights using gradients from backward pass
          learning_rate: the learning rate used in the gradient descent
        """
        self.weights = self.weights - learning_rate * self.dW
        self.bias = self.bias - learning_rate * self.db

        
    def outputDimension(self):
        """
        Returns the output dimension for the next layer
        """
        return len(self.bias)

    
    def __repr__(self):
        """
        Used to print a pretty summary of the layer
        """
        act = 'none' if self.activation == '' else self.activation
        return f'Dense layer (nx={self.weights.shape[1]}, nh={self.weights.shape[0]}, activation={act})'



### Alright, one down and one to go.
Next we will create a class that will combine the layers. It will also hold the loss function and has to calculate the gradient of the loss. For convenience, we will also add a wrapper to add layers and add a way to pretty print our model.

First we start again with the constructor, which has two options, the loss function to be used, and the randomMultiplier used for new layers. The loss function are again created in a function reference and are called using wrapper functions.

The model starts with no layers (empty).

```python
class NeuralNetwork:
    def __init__(self, loss='cross-entropy', randomMultiplier = 0.01):
        self.layers=[]
        self.randomMultiplier = randomMultiplier
        if loss=='cross-entropy':
            self.lossFunction = self.crossEntropyLoss
            self.lossBackward = self.crossEntropyLossGrad
        elif loss=='mean-square-error':
            self.lossFunction = self.meanSquareError
            self.lossBackward = self.meanSquareErrorGrad
        else:
            raise LossFunctionNotDefined
        self.loss=loss
```


The next function is a helper function to add layers to the model. You need to give the inputDimension, i.e. the number of input features of the first layer. For the second and further, it will look for the previous layer and use that as the input dimension.

You have to specify the number of neurons in the layer (units) and which activation function to use.

```python
    def addLayer(self, inputDimension=None, units=1, activation=''):
        if (inputDimension is None):
            if (len(self.layers)==0):
                raise InputDimensionNotCorrect
            inputDimension=self.layers[-1].outputDimension()
        layer = DenseLayer(inputDimension, units, activation, randomMultiplier= self.randomMultiplier)
        self.layers.append(layer)
```

Next we define the loss functions and their differentials. Feel free to check if the differentials are correct. the cost method is a wrapper to cal the cost in the training loop later.

```python
    def crossEntropyLoss(self, Y, A, epsilon=1e-15):
        m = Y.shape[1]
        loss = -1 * (Y * np.log(A + epsilon) + (1 - Y) * np.log(1 - A + epsilon))
        cost = 1 / m * np.sum(loss)
        return np.squeeze(cost)
            
    def crossEntropyLossGrad(self, Y, A):
        dA = -(np.divide(Y, A) - np.divide(1 - Y, 1 - A))
        return dA
    
    
    def meanSquareError(self, Y, A):
        loss = np.square(Y - A)
        m = Y.shape[1]
        cost = 1 / m * np.sum(loss)
        return np.squeeze(cost)
    
    def meanSquareErrorGrad(self, Y, A):
        dA = -2 * (Y - A)
        return dA

    
    def cost(self, Y, A):
        return self.lossFunction(Y, A)
```

The forward, backward, and update method are quite similar as they loop over the layers:

```python
    def forward(self, X):
        x = np.copy(X)
        for layer in self.layers:
            x = layer.forward(x)
        return x
            
    
    def backward(self, A, Y):
        dA = self.lossBackward(Y, A)
        for layer in reversed(self.layers):
            dA = layer.backward(dA)
    
    
    def update(self, learning_rate=0.01):
        for layer in self.layers:
            layer.update(learning_rate)
```

the backward pass first hast to calculate the first dA, which is de gradient between the activation and the real values. This is then looped through the layers to calculate all gradients.


The last two methods are again for pretty printing and one for trainable parameter counting. Nothing special.

### The complete NeuralNetwork class

In [ ]:
class NeuralNetwork:
    """
    Neural Network structure that holds our layers
    """
    
    def __init__(self, loss='cross-entropy', randomMultiplier = 0.01):
        """
        Constructor:
          loss: the loss function. Two are defined:
             - 'cross-entropy' and 'mean-square-error'
          randomMultiplier: multiplier applied to the random weights during initialization
        """
        self.layers=[]
        self.randomMultiplier = randomMultiplier
        if loss=='cross-entropy':
            self.lossFunction = self.crossEntropyLoss
            self.lossBackward = self.crossEntropyLossGrad
        elif loss=='mean-square-error':
            self.lossFunction = self.meanSquareError
            self.lossBackward = self.meanSquareErrorGrad
        else:
            raise LossFunctionNotDefined
        self.loss=loss


    def addLayer(self, inputDimension=None, units=1, activation=''):
        """
        Adds a Dense layer to the network:
          inputDimension: required when it is the first layer. otherwise takes dimensions of previous layer.
          units: number of neurons in the layer
          activation: activation function: valid choices are: 'sigmoid', 'tanh', 'relu', ''
        """
        if (inputDimension is None):
            if (len(self.layers)==0):
                raise InputDimensionNotCorrect
            inputDimension=self.layers[-1].outputDimension()
        layer = DenseLayer(inputDimension, units, activation, randomMultiplier= self.randomMultiplier)
        self.layers.append(layer)

    def crossEntropyLoss(self, Y, A, epsilon=1e-15):
        """
        Cross Entropy loss function
          Y: true labels
          A: final activation function (predicted labels)
          epsilon: small value to make minimize chance for log(0) error
        """
        m = Y.shape[1]
        loss = -1 * (Y * np.log(A + epsilon) + (1 - Y) * np.log(1 - A + epsilon))
        cost = 1 / m * np.sum(loss)
        return np.squeeze(cost)
            
    def crossEntropyLossGrad(self, Y, A):
        """
        Cross Entropy loss Gradient
          Y: true labels
          A: final activation function (predicted labels)
        """
        dA = -(np.divide(Y, A) - np.divide(1 - Y, 1 - A))
        return dA
    
    
    def meanSquareError(self, Y, A):
        """
        Mean square error loss function
          Y: true labels
          A: final activation function (predicted labels)
        """
        loss = np.square(Y - A)
        m = Y.shape[1]
        cost = 1 / m * np.sum(loss)
        return np.squeeze(cost)
    
    def meanSquareErrorGrad(self, Y, A):
        """
        Mean square error loss gradient
          Y: true labels
          A: final activation function (predicted labels)
        """
        dA = -2 * (Y - A)
        return dA

    
    def cost(self, Y, A):
        """
        Cost function wrapper
          Y: true labels
          A: final activation function (predicted labels)
        """
        return self.lossFunction(Y, A)

        
    def forward(self, X):
        """
        Forward pass through the whole model.
          X: input vector
        """
        x = np.copy(X)
        for layer in self.layers:
            x = layer.forward(x)
        return x
            
    
    def backward(self, A, Y):
        """
        backward pass through the whole model
          Y: true labels
          A: final activation function (predicted labels)
        """
        dA = self.lossBackward(Y, A)
        for layer in reversed(self.layers):
            dA = layer.backward(dA)
    
    
    def update(self, learning_rate=0.01):
        """
        Update weights and do a step of gradient descent for the whole model.
          learning_rate: learning_rate to use
        """
        for layer in self.layers:
            layer.update(learning_rate)
    
    
    def __repr__(self):
        """
        Pretty print the model
        """
        layrepr = ['  ' + str(ix+1)+' -> ' + str(x) for ix, x in enumerate(self.layers)]
        return '[\n' + '\n'.join(layrepr) + '\n]'
   
    
    def numberOfParameters(self):
        """
        Print number of trainable parameters in the model
        """
        n = 0
        for layer in self.layers:
            n += np.size(layer.weights) + len(layer.bias)
        print(f'There are {n} trainable parameters in the model.')
            

Alright, the classes are done. Now we need to put these classes to a test. While it looks quite ordered, a small error in one of the differentials can make our whole system useless. Therefore, we will test in small steps in the next section. Looking forward for the spin!

## 1. Does it do what it is supposed to do?

Nothing is more ennoying than waiting for nothing. Therefore, it is important to first test the easy things instead of training for hours to see that you made a mistake in the loss function. Let us repeat the experiments from last time

### Linear regression: 25 all over again!

We created an array with a couple of input values $X$. Next we supplied $Y$ using the super duper complicated formula $y = 2x + 1$. In Numpy this looks like this:

In [ ]:
import numpy as np

In [ ]:
X = np.arange(-2, 5, 1).reshape([1, 7]) # Columns as examples
Y = 2 * X + 1

Now we will build our model which will try to find our difficult formula and match the true value of 25. For this we will initiate our fresh class and set the loss to mean-square-error. Next we will add a single layer, with a single neuron and no activation function. Lets also try our pretty print :-)

In [ ]:
np.random.seed(1)

model = NeuralNetwork(loss='mean-square-error')
model.addLayer(inputDimension=1, units=1, activation='')
model

Now we will need our training loop again. It will look very familiar to our logistic regression model from last time:

In [ ]:
num_iterations = 1000
for ix in range(num_iterations):
    A = model.forward(X)
    model.backward(A, Y)
    model.update()
    if ix % 100 == 0:
        print('cost:', model.cost(Y, A)) 

The cost decreases as expected and due to the long training the value is close to machine precision. Would we again find 25 when we use the forward?

In [ ]:
model.forward(12)

Of course we do!

### A go at our Titanic dataset

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./data/titanic/train_data.csv')
df = df.drop(['Unnamed: 0', 'PassengerId'], axis=1)
Y = df['Survived'].to_numpy().reshape([1, -1])
X = df.iloc[:,1:].to_numpy().T

In [ ]:
X.shape

In [ ]:
Y.shape

We need to change the model to have 14 input features and of course the sigmoid activation function. The loss function will be cross-entropy, which is the default, so we do not need to specify it.

In [ ]:
np.random.seed(1)

model = NeuralNetwork()
model.addLayer(inputDimension=14, units=1, activation='sigmoid')
model

Last time, we had an train accuracy of 80%. Can we match this as well?

In [ ]:
def roundValue(A):
    return np.uint8( A > 0.5)

def accuracy(yhat, Y):
    return round(np.sum(yhat==Y) / len(yhat.flatten()) * 1000) / 10

In [ ]:
num_iterations = 8000
for ix in range(num_iterations):
    A = model.forward(X)
    model.backward(A, Y)
    model.update()
    if ix % 1000 == 0:
        yhat = roundValue(A)
        print('cost:', model.cost(Y, A), f'\taccuracy: {accuracy(yhat, Y)}%')     

Of course we can!

### Q1: We will now implement customization via Keras as the examples above are specific to their respective datasets and are for example only to work through for learning purposes only. **They will not properly run if customized**

In [ ]:
# Prerequisite library imports
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

# Let's reimport our data
df = pd.read_csv('./data/titanic/train_data.csv')
X = df.drop(['Unnamed: 0', 'PassengerId'], axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)
display(X_train.shape)
display(y_train.shape)

# Let's initialize our model
model = Sequential() # Initialising the ANN

In [ ]:
import seaborn as sns
sns.pairplot(X)

### 1.1) Above we use the Keras libraries to build the model. Here we want to implement a form of scaling to your data either through minmax normalization or z-score standardization using the sklearn.preprocessing libraries (MinMaxScalar or StandardScaler). Justify why you chose one over the other. If the data is already scaled, you still must choose to normalize or standardize. (10 points) 

### 1.2) Using the example for traindata above create a model using different activation functions by setting MYACTIVATIONFXN: (10 points) 

```python
# Hint! Start with model.add(Dense(units = 16, activation = 'relu', input_dim = ?))
# Make sure the input_dim parameter is set to the number of features in your X matrix.
MYACTIVATIONFXN = 'SOMEFXN'
model.add(Dense(units = 14, activation = MYACTIVATIONFXN, input_dim = ?))
```


In [ ]:
# If you decide to initially use a signmoid, make sure the number of units matches the number of targets
# in this case we only have 1 target so for sigmoid you need to set units to 1

### Now lets compile our model using the function compile
### Here we will use rmsprop as an optimizer and binary crossentropy as our loss function

In [ ]:
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy')


### Here we will run our ANN using the fit function using a batch size of 1 and 10 epochs (In the examples above iterations is used instead of epochs). Feel free to play with batch_size and epochs.

In [ ]:
model.fit(X_train.astype('float'), y_train, batch_size = 1, epochs = 10)

In [ ]:
model.weights

### 1.3) How does the error (in terms of precission and recall) differ between your model and the example? Write in one paragraph or less how the error differs and why. (5 points) 

In [ ]:
# Hint! Use the predict function and threshold your results. 0.5 is reasonable.
# Please see the BCC jupyter notebook to see how to do this

## 2) Complex fit of flowers

The cool stuf starts with more complex functions. The [Deep learning course from Andrew Ng](https://www.coursera.org/learn/neural-networks-deep-learning?specialization=deep-learning) show a way to predict [Rose-functions](https://en.wikipedia.org/wiki/Rose_(mathematics)) using a model with multiple nodes. Lets try that as well!

First we need to import the data:

In [ ]:
import numpy as np
data = np.load('./data/rose/rose.npz')
X, Y = data['X'], data['Y']

To give a feel how it looks, we will first plot the rose, which has 7 petals:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
plt.scatter(X[0,:], X[1,:], c=Y.flatten())

We discussed before that a logistic regression model, like we have defined in our previous example, can only divide in a line. Lets try this out, as the data is clearly not defined in a line.

In [ ]:
np.random.seed(1)

model = NeuralNetwork()
model.addLayer(inputDimension=2, units=1, activation='sigmoid')
model

In [ ]:
num_iterations = 1000
for ix in range(num_iterations):
    A = model.forward(X)
    model.backward(A, Y)
    model.update(learning_rate=1.2)
    if ix % 1000 == 0:
        print('cost:', model.cost(Y, A)) 

A short piece of code to generate a contour plot and our Rose

In [ ]:
def testModel(X, Y, h=0.1, f=1.05):
    r = X.max()
    xmesh, ymesh = np.meshgrid(np.arange(-r*f, r*f+h, h), np.arange(-r*f, r*f+h, h))
    Z = model.forward(((np.c_[xmesh.ravel(), ymesh.ravel()]).T))
    Z = (Z > 0.5) * 1
    Z = Z.T.reshape(xmesh.shape)
    plt.contourf(xmesh, ymesh, Z, cmap=plt.cm.OrRd)
    plt.scatter(X[0, :], X[1, :], c=Y.flatten(), cmap=plt.cm.OrRd)
testModel(X, Y)

Definitely not a great fit and clearly a line. Now lets add another layer with four units to the model, with the last layer being the same sigmoid layer. The activation function for this model we keep similar to the one from Andrew, which was a tanh.

In [ ]:
np.random.seed(1)

model = NeuralNetwork()
model.addLayer(inputDimension=2, units=4, activation='tanh')
model.addLayer(units=1, activation='sigmoid')
model

In [ ]:
num_iterations = 4000
for ix in range(num_iterations):
    A = model.forward(X)
    model.backward(A, Y)
    model.update(learning_rate=1.2)
    if ix % 1000 == 0:
        print('cost:', model.cost(Y, A)) 

In [ ]:
testModel(X,Y)

This is something that really amazes me. Just a tiny layer more and we have the power to learn this much more complex function. Really great stuff!

### Q2: We will now implement customization via Keras as the examples above are specific to their respective datasets and are for example only to work through for learning purposes only. They will not properly run if customized

In [ ]:
import numpy as np
data = np.load('./data/rose/rose.npz')
X, y = data['X'].transpose(), data['Y'].transpose()
display(X.shape)
display(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)
display(X_train.shape)
display(y_train.shape)

# Let's initialize our model
model = Sequential() # Initialising the ANN

### 2.1) Using the example above, try different number of nodes(units) and different activation functions. How does your loss change? (10 points) 

### 2.2) Calculate your new error for 2 different models using classification report. Also, using the metrics, explain why you see the same or why you see a different error. (10 points) 

### 2.3) Choose your best model! Now plot the new results using the plotting example shown above but using our newly trained best/coolest model. (10 points) 

## 3) Cats vs not cats

In the same course from Andrew, they pointed out that there is a terrible shortage of cat-detectors on the internet. Therefore, our class should detect cats in style! Lets download the same dataset. For details, please check out Andrew's Course, which is big fun!

In [ ]:
data = np.load('./data/cats/cats.npz')
X_train, y_train = data['Xtrain'], data['Ytrain']
X_test, y_test = data['Xtest'], data['Ytest']
display(X_train.shape)
display(y_train.shape)

Same functions as before.

In [ ]:
def roundValue(A):
    return np.uint8( A > 0.5)

def accuracy(yhat, Y):
    return round(np.sum(yhat==Y) / len(yhat.flatten()) * 1000) / 10

The model will be the same as the two layer model from the course. first layer has 7 units and a Relu activation function. Second layer is the sigmoid cat/no-cat layer.

In [ ]:
nx = X_train.shape[0]

np.random.seed(1)

model = NeuralNetwork()
model.addLayer(inputDimension=nx, units=7, activation='relu')
model.addLayer(units=1, activation='sigmoid')
model

In [ ]:
num_iterations = 1000
for ix in range(1, num_iterations+1):
    A = model.forward(X_train)
    model.backward(A, y_train)
    model.update(learning_rate=0.03)
    if ix % 100 == 0:
        yhat_test = roundValue(A)
        print('cost:', model.cost(y_train, A), f'\taccuracy: {accuracy(yhat_test, y_train)}%')     

Amazing accuracy. But we of course all know we should check this with a proper test set:

In [ ]:
A = model.forward(X_test)
yhat_test = roundValue(A)
acc = accuracy(yhat_test, y_test)
acc

An accuracy of 74% is definitely not bad for such a simple model which only works on pixel values. For the fun of it, lets look at some samples:

In [ ]:
n = 3

imgs = X_test.T.reshape([50, 64, 64, 3])

fig, ax = plt.subplots(1, n, figsize=(16,8))
for ix in range(n):
    num = np.random.randint(imgs.shape[0])
    ax[ix].imshow(imgs[num])
    if yhat.flatten()[num] == 0:
        ax[ix].set_title('This is clearly not a cat')
    else:
        ax[ix].set_title('Yup! it\'s a cat')


### Q3: Your turn! Let's do this!!!

In [ ]:
data = np.load('./data/cats/cats.npz')
X_train, y_train = data['Xtrain'].transpose(), data['Ytrain'].transpose()
X_test, y_test = data['Xtest'].transpose(), data['Ytest'].transpose()
display(X_train.shape)
display(y_train.shape)

# Let's initialize our model
model = Sequential() # Initialising the ANN

### 3.1) Same as before, build a new model with different number of hidden layers, nodes and activation functions. Describe reason for any similarity or difference (30 points) 

In [ ]:
# Try using different iterations using a simple layout like above. 10, 100, 1000 epochs. 
# What happens with your loss?

In [ ]:
# Try using different layers and activation function with different number of nodes

In [ ]:
# What happens when you add convolutional layers? What happens to our training loss?

### 3.2) Calculate your accuracy (10 points) 

In [ ]:
# Hint! Use the predict function and threshold your results. 0.5 is reasonable.
# In your classification report since we are only predicting cats you will need to set the parameter labels
# labels=np.unique(yhat_test)

### 3.3) Calculate your precision and recall manually (10 points) 

In [ ]:
# Recall calculation

In [ ]:
# Precision calculation

### Let's plot!!!

In [ ]:
n = 3
imgs = X_test.reshape([50, 64, 64, 3])
fig, ax = plt.subplots(1, n, figsize=(16,8))
for ix in range(n):
    num = np.random.randint(imgs.shape[0])
    ax[ix].imshow(imgs[num])
    if yhat_test[num] == 0:
        ax[ix].set_title('This is clearly not a cat')
    else:
        ax[ix].set_title('Yup! it\'s a cat')

## Round up!

I hope you all had fun, writing your own ANN. In my opinon, writing these things from the ground up is the best way to learn how it actually works. I hope that you see that these systems are not magical, but simple matrix multiplications, unfortunately just a very lot of them. The most difficult part is of course the back propagation, where we need to calculate the gradients. Our simple ANNs are quite doable, but adding more different layers to them, can make it a bit more cumbersome. Still the essence is very similar to what we have done today.

My suggestion is to play around with these structures, rewrite parts of them, or even better, write your own from scratch!


Please let me know if you have any comments!

## Apendix

### Generating Rose Data

In [ ]:
def generateRoseData():
    k=7
    pointPerPetal = 100
    cutOff = 0.1
    r = 4

    theta = np.linspace(0,np.pi, pointPerPetal * k)
    xx = r * np.cos(k * theta) * np.cos(theta)
    yy = r * np.cos(k * theta) * np.sin(theta)
    cc = [np.ones(pointPerPetal) if ix % 3 == 0 else np.zeros(pointPerPetal) for ix in np.arange(k)]
    cc = np.roll(np.hstack(cc).astype(np.uint8), -pointPerPetal//2)
    x = xx[(xx**2 + yy**2)**0.5 > cutOff]
    y = yy[(xx**2 + yy**2)**0.5 > cutOff]
    col = cc[(xx**2 + yy**2)**0.5 > cutOff]
    X = np.vstack([x,y])
    Y = np.copy(col).reshape([1, -1])
    return X, Y
X, Y = generateRoseData()
np.savez_compressed('./data/rose/rose.npz', X=X, Y=Y)

### Processing Andrews CatvNotCat data

In [ ]:
# If you get an error here, install h5py via pip3 install h5py
import h5py

In [ ]:
# Data downloaded from:
# https://github.com/ridhimagarg/Cat-vs-Non-cat-Deep-learning-implementation
def processCatData():
    train_dataset = h5py.File("./data/cats/train_catvnoncat.h5", mode='r')
    Xtrain = np.array(train_dataset["train_set_x"])
    Y_train = np.array(train_dataset["train_set_y"])
    test_dataset = h5py.File("./data/cats/test_catvnoncat.h5", mode='r')
    Xtest = np.array(test_dataset["test_set_x"])
    Y_test = np.array(test_dataset["test_set_y"])
    X_train = Xtrain / 255
    X_test = Xtest / 255
    X_train = X_train.reshape(209, -1).T
    Y_train = Y_train.reshape(-1, 209)
    X_test = X_test.reshape(50, -1).T
    Y_test = Y_test.reshape(-1, 50)
    return X_train, X_test, Y_train, Y_test
Xtrain, Xtest, Ytrain, Ytest = processCatData()
np.savez_compressed('./data/cats/cats.npz', Xtrain=Xtrain, Xtest=Xtest, Ytrain=Ytrain, Ytest=Ytest)

# Credits
#### Edwin Solares - Conversion to google colab, conversion to Keras and preprocessing data to work with Kears
#### Dennis Bakhuis - May the Fourth (be with you) 2020
https://linkedin.com/in/dennisbakhuis/ \
https://github.com/dennisbakhuis


### Exercise 4 - Collaborative Statement (5 points) 
#### You must fill this out even if you worked alone to get credit.

It is mandatory to include a Statement of Collaboration in each submission, that follows the guidelines below.
Include the names of everyone involved in the discussions (especially in-person ones), and what was discussed.
All students are required to follow the academic honesty guidelines posted on the course website. For
programming assignments in particular, I encourage students to organize (perhaps using Piazza) to discuss the
task descriptions, requirements, possible bugs in the support code, and the relevant technical content before they
start working on it. However, you should not discuss the specific solutions, and as a guiding principle, you are
not allowed to take anything written or drawn away from these discussions (no photographs of the blackboard,
written notes, referring to Piazza, etc.). Especially after you have started working on the assignment, try to restrict
the discussion to Piazza as much as possible, so that there is no doubt as to the extent of your collaboration.